# Install Required Libraries

**QUERY**: "Could you inform me about the prices of earbuds?"

**ANSWER**

**Before RAG**:

"Certainly! TechStyle Global offers a variety of earbuds to suit different preferences and budgets. Here are some options:


1. **TechStyle Basic Earbuds**: These are our most affordable earbuds, priced at $29.99. They offer decent sound quality and are great for everyday use.


2. **TechStyle Pro Earbuds**: For those looking for a bit more performance, the Pro Earbuds are priced at $49.99. They feature active noise cancellation and a more immersive sound experience.


3. **TechStyle Elite Earbuds**: If you're looking for the highest quality, the Elite Earbuds are available for $79.99. They come with superior sound quality, long battery life, and a sleek design.


Please note that prices are subject to change and may vary based on location and sales. For the most accurate and up-to-date pricing, you can visit our website or contact our customer service."

**USING RAG**:

"Certainly! TechStyle offers two types of earbuds. The TechStyle Wireless Earbuds are priced at $69.99 and feature a 24-hour battery life, touch controls, water resistance, and noise isolation. The TechStyle Over-Ear Headphones are slightly more expensive at \$89.99 and include Bluetooth 5.0, a 30-hour battery life, a foldable design, and a carrying case."

In [ ]:
!pip install -q transformers gradio langchain langchain-community langchain-chroma

# Set Up Hugging Face API Token

In [ ]:
from google.colab import userdata
HF_TOKEN = userdata.get("HF_TOKEN")

# Define Hugging Face API Query Function

In [ ]:
import requests

API_URL = "https://api-inference.huggingface.co/models/microsoft/Phi-3-mini-4k-instruct"
headers = {"Authorization": f"Bearer {HF_TOKEN}"}

def query_hf_model(inputs, context="", max_new_tokens=500, temperature=0.2):

    inputs = f"""<|system|>
You are TechStyle Phi, the digital assistant for TechStyle Global, \
dedicated to helping customers with their inquiries about the company. \
Always provide accurate information about TechStyle Global, \
its products, and shipping details.{context}<|end|>
<|user|>
{inputs}<|end|>
<|assistant|>
"""

    payload = {
        "inputs": inputs,
        "parameters": {
            "max_new_tokens": max_new_tokens,
            "temperature": temperature,
            "do_sample": True,
            "return_full_text": False,
        }
    }

    try:
        response = requests.post(API_URL, headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()

        if isinstance(result, list) and "generated_text" in result[0]:
            return result[0]["generated_text"]

        return "Error: No generated text found in the response."

    except requests.exceptions.RequestException as e:
        return f"Error: {e}"

#  Generate Response Without Retrieval (Before RAG)

In [ ]:
input_text = "Could you inform me about the prices of earbuds?"
response = query_hf_model(input_text)
print("Response without RAG:", end="\n\n")
print(response)

Response without RAG:

Certainly! TechStyle Global offers a variety of earbuds to suit different preferences and budgets. Here are some options:


1. **TechStyle Basic Earbuds**: These are our most affordable earbuds, priced at $29.99. They offer decent sound quality and are great for everyday use.


2. **TechStyle Pro Earbuds**: For those looking for a bit more performance, the Pro Earbuds are priced at $49.99. They feature active noise cancellation and a more immersive sound experience.


3. **TechStyle Elite Earbuds**: If you're looking for the highest quality, the Elite Earbuds are available for $79.99. They come with superior sound quality, long battery life, and a sleek design.


Please note that prices are subject to change and may vary based on location and sales. For the most accurate and up-to-date pricing, you can visit our website or contact our customer service.


# Load and Process Text Files for Retrieval

In [ ]:
def load_text_files():
    data = {}
    for filename in ["company.txt", "products.txt", "shipping.txt"]:
        with open(f"/content/{filename}", "r") as file:
            data[filename] = file.read()
    return data

text_data = load_text_files()
print("Loaded Text Data:", text_data)

Loaded Text Data: {'company.txt': "# TechStyle Global - Company Overview\n\nFounded in 2015, TechStyle Global is a leading e-commerce marketplace specializing in consumer electronics, fashion, and lifestyle products. Our platform connects millions of customers worldwide with quality products at competitive prices.\n\n## Our Story\nTechStyle began as a small online electronics store in San Francisco, founded by tech entrepreneurs Sarah Chen and Marcus Rodriguez. Their vision was to create a seamless shopping experience that bridges the gap between technology and lifestyle products. Within five years, we expanded globally, now serving customers in over 50 countries.\n\n## Mission Statement\nTo provide accessible, innovative, and sustainable shopping solutions that enhance people's daily lives through technology and style.\n\n## Core Values\n- Customer First: Every decision we make starts with our customers\n- Innovation: Constantly improving our platform and services\n- Sustainability: C

# Split Text Data into Chunks

In [ ]:
from langchain.text_splitter import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on,
                                               strip_headers = False)

def split_text_with_headers(text_data):
    splits = []
    for filename, text in text_data.items():

        chunk_splits = markdown_splitter.split_text(text)
        splits.extend(chunk_splits)

    return splits

md_header_splits = split_text_with_headers(text_data)
print("Split Text Data:", md_header_splits)


Split Text Data: [Document(metadata={'Header 1': 'TechStyle Global - Company Overview'}, page_content='# TechStyle Global - Company Overview  \nFounded in 2015, TechStyle Global is a leading e-commerce marketplace specializing in consumer electronics, fashion, and lifestyle products. Our platform connects millions of customers worldwide with quality products at competitive prices.'), Document(metadata={'Header 1': 'TechStyle Global - Company Overview', 'Header 2': 'Our Story'}, page_content='## Our Story\nTechStyle began as a small online electronics store in San Francisco, founded by tech entrepreneurs Sarah Chen and Marcus Rodriguez. Their vision was to create a seamless shopping experience that bridges the gap between technology and lifestyle products. Within five years, we expanded globally, now serving customers in over 50 countries.'), Document(metadata={'Header 1': 'TechStyle Global - Company Overview', 'Header 2': 'Mission Statement'}, page_content="## Mission Statement\nTo pro

# Create and Initialize Chroma Vector Store

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector_store = Chroma.from_documents(
    documents=md_header_splits,
    embedding=embeddings,
    persist_directory="/content/vector_store"
)

retriever = vector_store.as_retriever(search_kwargs={"k": 3})

# Retrieve and Generate Response (Using RAG)

In [ ]:
def retrieve_and_generate(question):

    docs = retriever.get_relevant_documents(question)

    for doc in docs:
      print(doc.page_content, end="\n\n")

    context = " ".join([doc.page_content for doc in docs])

    context = f""" Please answer the questions about TechStyle, its products and shipping details \n
using this information only: {context}."""

    return query_hf_model(inputs=question, context=context)


In [ ]:
query = "Could you inform me about the prices of earbuds?"
result = retrieve_and_generate(query)
print("# Response after RAG:", end="\n\n")
print(result)

### Audio
- TechStyle Wireless Earbuds
- Price: $69.99
- 24-hour battery
- Touch controls
- Water resistant
- Noise isolation  
- TechStyle Over-Ear Headphones
- Price: $89.99
- Bluetooth 5.0
- 30-hour battery
- Foldable design
- Carrying case included

### Bundles
- Work From Home Bundle
- Price: $1499.99
- UltraBook Air 13"
- Wireless Mouse
- External Monitor
- Desk Stand
- Webcam  
- Smart Home Starter Bundle
- Price: $299.99
- SmartCam Pro
- Smart Light Kit
- Smart Plug (2-pack)
- Voice Assistant

## Wearable Technology  
### Fitness Trackers
- FitStyle Pro
- Price: $149.99
- Heart rate monitoring
- Sleep tracking
- 7-day battery life
- Water resistant  
- SportBand Lite
- Price: $79.99
- Step counting
- Basic sleep tracking
- 14-day battery life
- 5 color options

# Response after RAG:

Certainly! TechStyle offers two types of earbuds. The TechStyle Wireless Earbuds are priced at $69.99 and feature a 24-hour battery life, touch controls, water resistance, and noise isolation. The 

#  Gradio Interface

In [ ]:
import gradio as gr

def gradio_interface(query):
    return retrieve_and_generate(query)

interface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(label="Hi, I'm TechStyle Phi. How can I help you today?"),
    outputs=gr.Textbox(label="TechStyle Phi")
)

interface.launch()